In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy.engine import Engine
from sqlalchemy import create_engine

base_path = "/media/bruno/Arquivos/Desenvolvimento/DataScienceSalaries"
load_dotenv(f"{base_path}/config/.env")

True

In [2]:
def get_engine() -> Engine:
    user = os.getenv("DB_USER")
    pw = os.getenv("DB_PASS")
    db = os.getenv("DB_NAME")
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")
    return create_engine(f"mysql+pymysql://{user}:{pw}@{host}:{port}/{db}")

In [12]:
def description() -> None:
    engine = get_engine()
    with engine.connect() as con:
        dataset = pd.read_sql("select * from kaggle.ds_salaries", con)
        print(dataset.columns)

        print("DISTRIBUIÇÃO DE CARGOS AVALIADOS")
        for job in dataset["job_title"].unique():
            total = list(dataset["job_title"].values).count(job)
            if total < 10:
                continue
            print(f"{job:{50}} {total} ({round(total * 100 / len(dataset), 2)}%)")


description()

Index(['id', 'work_year', 'experience_level', 'employment_type', 'job_title',
       'salary_usd', 'employee_residence', 'remote_ratio', 'company_location',
       'company_size'],
      dtype='object')
DISTRIBUIÇÃO DE CARGOS AVALIADOS
Data Scientist                                     143 (23.56%)
Machine Learning Engineer                          41 (6.75%)
Data Analyst                                       97 (15.98%)
Data Engineer                                      132 (21.75%)
Research Scientist                                 16 (2.64%)
Data Science Manager                               12 (1.98%)
Data Architect                                     11 (1.81%)
